# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [6]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [7]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [8]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [10]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [11]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [12]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [13]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [14]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [15]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [16]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [17]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [18]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


2024-05-22 12:19:35,331 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:19:35,332 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:19:35,333 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')




> Entering new CrewAgentExecutor chain...


2024-05-22 12:19:37,581 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:19:37,582 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:19:37,582 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


Thought: The first step is to gather any available instructions or files that might contain relevant information about DeepLearningAI or general guidelines I need to follow to complete this task.

Action: List files in directory
Action Input: {}
 

File paths: 
-./instructions/enterprise_solutions_framework.md
- ./instructions/tech_startups_outreach.md
- ./instructions/small_business_engagement.md



...



> Entering new CrewAgentExecutor chain...
I need to gather relevant information about DeepLearningAI, including their recent product launch, company culture, values, and how CrewAI's solutions can support their goals. To create a compelling and personalized outreach campaign, I'll start by searching the internet for recent news and updates about DeepLearningAI.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI recent product launch news"}

2024-05-22 12:20:18,012 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:20:18,013 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:20:18,013 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


 


Search results: Title: DeepLearning.AI: Start or Advance Your Career in AI
Link: https://www.deeplearning.ai/
Snippet: Learn the skills to start or advance your AI career | World-class education | Hands-on training | Collaborative community of peers and mentors.
---
Title: Google's Troubled Gemini Launch, OpenAI's Next Act, and more
Link: https://www.deeplearning.ai/the-batch/issue-238/
Snippet: An upstart chip company dramatically accelerates pretrained large language models. What's new: Groq offers cloud access to Meta's Llama 2 and ...
---
Title: The Batch | DeepLearning.AI | AI News & Insights
Link: https://www.deeplearning.ai/the-batch/
Snippet: The Batch AI News and Insights: In the last couple of days, Google announced a doubling of Gemini Pro 1.5's input context window from 1 million to 2 million ...
---
Title: OpenAI | The Batch | AI News & Insights - DeepLearning.AI
Link: https://www.deeplearning.ai/the-batch/tag/openai/
Snippet: OpenAI Expands Platform Play: OpenAI relea

2024-05-22 12:20:26,293 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:20:26,309 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:20:26,310 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 12:20:35,444 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

Thought: The sentiment analysis of the first email draft returned a positive result, indicating that the tone is engaging and appropriate. Now, I will proceed to draft the second email.

Here is a draft of the second email:

---

Subject: Transforming AI Education Together

Dear Dr. Andrew Ng,

Congratulations on the successful launch of "Generative AI for Everyone." This is a significant milestone in the journey to make AI accessible to all, and it's a testament to DeepLearningAI's commitment to innovation and education.

At CrewAI, we are dedicated to pushing the boundaries of what AI can achieve in the educational sector. Our suite of AI-driven tools can complement your new course by enhancing interactive learning experiences, providing personalized feedback, and offering in-depth analytics to understand learner behavior and outcomes.

We see a strong alignment between DeepLearningAI's vision and our expertise in AI technologies. Let's explore how we can work together to create an e

- Display the final result as Markdown.

In [20]:
from IPython.display import Markdown
Markdown(result)

Here are the personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng:

---

**Email 1:**

**Subject:** Empowering DeepLearningAI's Generative AI Journey

Dear Dr. Andrew Ng,

I hope this message finds you well. I was thrilled to hear about the launch of "Generative AI for Everyone" on Coursera. It's inspiring to see DeepLearningAI continue to lead the way in democratizing AI education and empowering learners globally.

At CrewAI, we share your passion for innovation and excellence in AI. Our advanced AI solutions are designed to support educational platforms like yours by enhancing scalability, improving user engagement, and providing robust analytics to track learner progress.

We believe there is a great synergy between DeepLearningAI's mission and our capabilities. I would love the opportunity to discuss how CrewAI can support your goals and help you further elevate the learning experience for your students.

Looking forward to the possibility of collaborating with you.

Best regards,

[Your Name]  
Lead Sales Representative  
CrewAI

---

**Email 2:**

**Subject:** Transforming AI Education Together

Dear Dr. Andrew Ng,

Congratulations on the successful launch of "Generative AI for Everyone." This is a significant milestone in the journey to make AI accessible to all, and it's a testament to DeepLearningAI's commitment to innovation and education.

At CrewAI, we are dedicated to pushing the boundaries of what AI can achieve in the educational sector. Our suite of AI-driven tools can complement your new course by enhancing interactive learning experiences, providing personalized feedback, and offering in-depth analytics to understand learner behavior and outcomes.

We see a strong alignment between DeepLearningAI's vision and our expertise in AI technologies. Let's explore how we can work together to create an even more impactful learning environment for your students.

Warm regards,

[Your Name]  
Lead Sales Representative  
CrewAI

---

These drafts are designed to resonate with DeepLearningAI's company culture and values, demonstrating a deep understanding of their business and needs.